In [ ]:
import pandas as pd
import nltk
nltk.download('punkt')
from preprocessing import get_word_sentence_counts
from format import change_model_names

In [ ]:
detection_df = pd.read_csv("../../data/detection/mgt_detection3.csv", encoding="utf-8")

In [ ]:
# get word and sentence counts for each text
detection_df["sentence_count"] = detection_df["Generation"].apply(lambda x: get_word_sentence_counts(str(x))[0])
detection_df["word_count"] = detection_df["Generation"].apply(lambda x: get_word_sentence_counts(str(x))[1])

In [ ]:
# calculate mean and std for word count and sentece count for each label
detection_df.groupby("label").agg({"sentence_count": ["mean", "std"], "word_count": ["mean", "std"]})

In [ ]:
df = pd.read_csv("../../data/data.csv", encoding="utf-8")

In [ ]:
df.reset_index(inplace=True)

In [ ]:
# add model to df from final_df according to generated_text
def get_models(detection_df, model_df):
    for index, row in detection_df.iterrows():
        idx = model_df.index[model_df['generated_text'] == row['Generation']]
        if idx.size > 0:
            idx = idx[0]
            detection_df.at[index, 'model'] = model_df.loc[idx, 'model']
        else:
            detection_df.at[index, 'model'] = "human"

    return detection_df

In [ ]:
detection_df = get_models(detection_df, df)

In [ ]:
# visualize only 2 decimals
def format_table(df):
    def custom_format(val):
        return f"{val:.2f}"
        
    for row_idx, row in enumerate(df.values):
        formatted_row = [custom_format(val)
                         for val in row]
        df.iloc[row_idx] = formatted_row
    return df

In [ ]:
custom_order = ['gpt-4', 'gpt-3.5-turbo', 'text-davinci-003', 'text-curie-001', 'text-babbage-001', 'vicuna-33b-v1.3', 'falcon-40b-instruct', 'Llama-2-70b-chat-hf', 'Mistral-7B-Instruct-v0.1', 'opt-iml-max-30b', 'human']
print(format_table(change_model_names(detection_df.groupby("model").agg({"sentence_count": ["mean", "std"], "word_count": ["mean", "std"]}).reindex(custom_order))).to_latex())